In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
import time

In [2]:
from openai import OpenAI
client = OpenAI()

In [19]:
r = pd.read_csv('../data/rmt_test_concepts.csv')
words = r['Word']
riddles = r['Riddle']
bfs_concepts = r['BFSConcepts']
dfs_concepts = r['DFSConcepts']
pq_concepts = r['PQConcepts']

In [24]:
openai_generations = []

In [29]:
for i in tqdm(range(3,103)):
  word = words[i]
  cp = pq_concepts[i]
  msg = f"Can you create a riddle for the word: {word} which touches upon these concepts: {cp[:4]} ?"
  completion = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      messages=[
        {"role": "user", "content": msg}
      ]
  )
  openai_generations.append(completion.choices[0].message.content)
  time.sleep(20)

100%|██████████| 100/100 [36:35<00:00, 21.95s/it]


In [22]:
prompts = []
for i in tqdm(range(len(words))):
  word = words[i]
  cp = eval(dfs_concepts[i])
  if len(cp)!=0:
    msg = f"Can you create a short riddle for the word: {word} which is based on following topics: {cp[:4]} ?"
  else:
    msg = f"Can you create a short riddle for the word: {word}?"
  prompts.append(msg)

  0%|          | 0/869 [00:00<?, ?it/s]

100%|██████████| 869/869 [00:00<00:00, 35596.33it/s]


In [23]:
prompts[:10]

['Can you create a short riddle for the word: Blood?',
 "Can you create a short riddle for the word: hope which is based on following topics: ['wish', 'faith', 'belief'] ?",
 'Can you create a short riddle for the word: ego?',
 'Can you create a short riddle for the word: water?',
 "Can you create a short riddle for the word: dream which is based on following topics: ['you dream', 'sleep', 'close your eyes'] ?",
 "Can you create a short riddle for the word: cat which is based on following topics: ['my lap', 'a bed', 'hunt mice'] ?",
 "Can you create a short riddle for the word: cloud which is based on following topics: ['sky', 'the sky', 'white'] ?",
 'Can you create a short riddle for the word: paparazzi?',
 "Can you create a short riddle for the word: dream which is based on following topics: ['you dream', 'sleep', 'close your eyes'] ?",
 'Can you create a short riddle for the word: wrestlemania?']

In [25]:
for i in tqdm(range(44)):
      ip_prompts = prompts[20*i:20*(i+1)]
      completion = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=ip_prompts,
            max_tokens=128
      )
      for resp in completion.choices:
            gen = resp.text
            openai_generations.append(gen)
      time.sleep(20)
      

  0%|          | 0/44 [00:00<?, ?it/s]

 86%|████████▋ | 38/44 [13:37<02:10, 21.74s/it]

In [ ]:
openai_gen = {
    "Words":words,
    "DFSConcepts":dfs_concepts,
    "GoldRiddle": riddles,
    "GenRiddle":openai_generations
}

In [ ]:
pd.DataFrame.from_dict(openai_gen).to_csv('gpt-turbo-gen-dfs.csv',index=False)